In [1]:
from bs4 import BeautifulSoup,Comment
import urllib.request
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
#from pandas import DataFrame as df

In [ ]:
#to get soup for 2020 team offense using Comment, used below
url = "https://www.pro-football-reference.com/years/2020/"
soup = BeautifulSoup(requests.get(url).content, 'html.parser')
table = soup.select_one('#all_team_stats').find_next(text=lambda t: isinstance(t, Comment))
table = BeautifulSoup(table, 'html.parser')



In [ ]:
#what the HTML looks like
table.prettify()[0:2500]

In [7]:
#function to rename some columns and columns that are missing titles from HTML table
def rename_cols(df):
    dict = {'Unnamed: 0_level_0':'TotOff',
            'Unnamed: 1_level_0':'TotOff',
            'Unnamed: 2_level_0':'TotOff',
            'Unnamed: 3_level_0':'TotOff',
            'Unnamed: 4_level_0':'TotOff',
            'Tot Yds & TO':'TotOff',
            'Tot Yds & TO':'TotOff',
            'Tot Yds & TO':'TotOff',
            'Unnamed: 8_level_0':'TotOff',
            'Unnamed: 9_level_0':'TotOff',
            'Unnamed: 25_level_0':'Score%',
            'Unnamed: 26_level_0':'TurnOver%',
            'Unnamed: 27_level_0':'EXP',}
        
    # call rename() method
    df.rename(columns=dict,inplace=True)
    return(df)


In [14]:
#for loop to get data for X years


year = 2020
#URL for website data
url = "https://www.pro-football-reference.com/years/"
urls = []
OFFdfs = []
for index in np.arange(10):
    #create URL for BeautifulSoup
    urls.append(url + str(year - index) +'/')
    years = year - index

count = 0
for index in urls:
    soup = BeautifulSoup(requests.get(index).content, 'html.parser')
    table = soup.select_one('#all_team_stats').find_next(text=lambda t: isinstance(t, Comment))
    table = BeautifulSoup(table, 'html.parser')    
    #create DF from html table, drop last 3 rows of data 
    OFFdf = pd.read_html(str(table))[0]
    curr_year = year - count
    OFFdf['Year'] = curr_year
    OFFdfs.append(OFFdf[:-3])
    count+=1


#initialize Master Team Offense Dataframe    
MasterOFF = OFFdfs[0]

for i in range(len(OFFdfs)-1):    
    #append DF to master, reset index to continuous
    MasterOFF = MasterOFF.append(OFFdfs[i+1],ignore_index = True)

#rename columns to useful index    
MasterOFF = rename_cols(MasterOFF)



In [ ]:
MasterOFF.info()

In [55]:
MasterOFF.head(5)

TotOff                                                                      \
      Rk                    Tm     G     PF     Yds     Ply  Y/P    TO    FL   
0    1.0     Green Bay Packers  16.0  509.0  6224.0   990.0  6.3  11.0   6.0   
1    2.0         Buffalo Bills  16.0  501.0  6343.0  1034.0  6.1  22.0  11.0   
2    3.0  Tampa Bay Buccaneers  16.0  492.0  6145.0  1017.0  6.0  17.0   5.0   
3    4.0      Tennessee Titans  16.0  491.0  6343.0  1031.0  6.2  12.0   5.0   
4    5.0    New Orleans Saints  16.0  482.0  6023.0  1045.0  5.8  17.0   9.0   

          ... Rushing             Penalties               Score% TurnOver%  \
    1stD  ...      TD  Y/A   1stD       Pen     Yds 1stPy    Sc%       TO%   
0  358.0  ...    16.0  4.8  114.0      84.0   684.0  28.0   49.7       5.6   
1  397.0  ...    16.0  4.2  119.0     102.0   941.0  38.0   49.4      11.8   
2  364.0  ...    16.0  4.1   82.0      84.0   715.0  44.0   47.8       8.9   
3  381.0  ...    26.0  5.2  142.0      86.0   783.0  36.0   47.9       7.2   
4  367.0  ...    30.0  4.6  147.0      98.0  1005.0  21.0   45.5       9.0   

      EXP  Year  
      EXP        
0  290.75  2020  
1  228.66  2020  
2  246.24  2020  
3  245.01  2020  
4  176.94  2020  

[5 rows x 29 columns]

In [ ]:
MasterOFF.keys()

In [ ]:
plt.figure(figsize=(10, 10))
#pointsfor = MasterOFF.TotOff.PF
MasterOFF.plot.scatter("Year", [('TotOff','PF')] , figsize=(15, 10))
plt.show()

In [71]:
#Get Team Defense data

year = 2020
#number of years
n=10
#URL for website data
url = "https://www.pro-football-reference.com/years/"
urls = []
DEFdfs = []
ADV_DEFdf = []
Team_Sacks = pd.DataFrame(['Tm','Sk'])
for index in np.arange(10):
    #create URL for BeautifulSoup
    urls.append(url + str(year - index) +'/opp.htm')
    years = year - index

    
#mask for advanced_defense table data
sacks = ['Tm','Sk']

#team_stats table does not include Sacks, so need passing table data and merge tables.
count = 0
for index in urls:
    soup = BeautifulSoup(requests.get(index).content, 'html.parser')
    table = soup.find('table', id = 'team_stats') 
    #table2 = soup.select('table', id = 'passing')
    table2 = soup.select_one('#all_passing').find_next(text=lambda t: isinstance(t, Comment))
    table2 = BeautifulSoup(table2, 'html.parser')
    #create DF from html table, drop last 3 rows of data 
    DEFdf = pd.read_html(str(table))[0]
    DEFdf.drop(DEFdf.tail(3).index, inplace = True)
    #rename each DF
    rename_cols(DEFdf)
    #sort alphabetically
    DEFdf.sort_values([('TotOff', 'Tm')],inplace=True)
    ADV_DEFdf = pd.read_html(str(table2))[0]
    ADV_DEFdf.drop(ADV_DEFdf.tail(3).index,inplace = True)
    #apply mask
    Team_Sacks = ADV_DEFdf[sacks].copy()
    #sort alphabetically
    Team_Sacks.sort_values(['Tm'],inplace = True)
    #intermediate step prior to appending data. merges team_stats and team sacks data 
    DEFdf_interm = pd.merge(DEFdf,Team_Sacks, left_on =[('TotOff', 'Tm')], right_on = 'Tm' )
    #append to list of team defense data
    curr_year = year - count
    DEFdf_interm['Year'] = curr_year    
    DEFdfs.append(DEFdf_interm)
    count+=1

#add in the year column
#for index in np.arange(10):
    #add Year to DF columns
#    curr_year = year - index
#    DEFdfs[index]['Year'] = curr_year

#initialize master DataFrame with first list of Team Defense data
MasterDEF = DEFdfs[0]

for i in range(len(DEFdfs)-1):    
    #append DF to master, reset index to continuous
    MasterDEF = MasterDEF.append(DEFdfs[i+1],ignore_index = True)
    
MasterDEF.drop('Tm',axis = 1,inplace=True)

MasterDEF.head(5)

/Users/scottstack/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/scottstack/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/scottstack/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/scottstack/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/scottstack/opt/anaconda3/

,"(TotOff, Rk)","(TotOff, Tm)","(TotOff, G)","(TotOff, PF)","(TotOff, Yds)","(TotOff, Ply)","(TotOff, Y/P)","(TotOff, TO)","(TotOff, FL)","(TotOff, 1stD)",...,"(Rushing, Y/A)","(Rushing, 1stD)","(Penalties, Pen)","(Penalties, Yds)","(Penalties, 1stPy)","(Score%, Sc%)","(TurnOver%, TO%)","(EXP, EXP)",Sk,Year
0,12.0,Arizona Cardinals,16.0,367.0,5631.0,1054.0,5.3,21.0,10.0,363.0,...,4.6,118.0,104.0,841.0,38.0,37.2,10.5,-82.04,48.0,2020
1,19.0,Atlanta Falcons,16.0,414.0,6374.0,1034.0,6.2,21.0,9.0,367.0,...,4.4,97.0,91.0,800.0,31.0,42.0,12.1,-148.58,29.0,2020
2,2.0,Baltimore Ravens,16.0,303.0,5276.0,1017.0,5.2,22.0,12.0,347.0,...,4.6,96.0,77.0,646.0,38.0,30.0,12.4,-17.69,39.0,2020
3,16.0,Buffalo Bills,16.0,375.0,5640.0,1025.0,5.5,26.0,11.0,351.0,...,4.6,125.0,86.0,728.0,27.0,39.4,15.3,-123.10,38.0,2020
4,18.0,Carolina Panthers,16.0,402.0,5761.0,1022.0,5.6,22.0,15.0,360.0,...,4.7,114.0,85.0,713.0,35.0,46.0,12.4,-125.54,29.0,2020


In [72]:
MasterDEF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   (TotOff, Rk)        320 non-null    float64
 1   (TotOff, Tm)        320 non-null    object 
 2   (TotOff, G)         320 non-null    float64
 3   (TotOff, PF)        320 non-null    float64
 4   (TotOff, Yds)       320 non-null    float64
 5   (TotOff, Ply)       320 non-null    float64
 6   (TotOff, Y/P)       320 non-null    float64
 7   (TotOff, TO)        320 non-null    float64
 8   (TotOff, FL)        320 non-null    float64
 9   (TotOff, 1stD)      320 non-null    float64
 10  (Passing, Cmp)      320 non-null    float64
 11  (Passing, Att)      320 non-null    float64
 12  (Passing, Yds)      320 non-null    float64
 13  (Passing, TD)       320 non-null    float64
 14  (Passing, Int)      320 non-null    float64
 15  (Passing, NY/A)     320 non-null    float64
 16  (Passing

In [73]:
MasterDEF.keys()

Index([      ('TotOff', 'Rk'),       ('TotOff', 'Tm'),        ('TotOff', 'G'),
             ('TotOff', 'PF'),      ('TotOff', 'Yds'),      ('TotOff', 'Ply'),
            ('TotOff', 'Y/P'),       ('TotOff', 'TO'),       ('TotOff', 'FL'),
           ('TotOff', '1stD'),     ('Passing', 'Cmp'),     ('Passing', 'Att'),
           ('Passing', 'Yds'),      ('Passing', 'TD'),     ('Passing', 'Int'),
          ('Passing', 'NY/A'),    ('Passing', '1stD'),     ('Rushing', 'Att'),
           ('Rushing', 'Yds'),      ('Rushing', 'TD'),     ('Rushing', 'Y/A'),
          ('Rushing', '1stD'),   ('Penalties', 'Pen'),   ('Penalties', 'Yds'),
       ('Penalties', '1stPy'),      ('Score%', 'Sc%'),   ('TurnOver%', 'TO%'),
               ('EXP', 'EXP'),                   'Sk',                 'Year'],
      dtype='object')

In [128]:
#for loop to get data for X years

#get standings inlcuding win-loss record

year = 2020
#URL for website data
url = "https://www.pro-football-reference.com/years/"
urls = []
AFCStandDfs = []
NFCStandDfs = []
for index in np.arange(10):
    #create URL for BeautifulSoup
    urls.append(url + str(year - index) +'/')
    years = year - index

count = 0
for index in urls:
    soup = BeautifulSoup(requests.get(index).content, 'html.parser')
    AFCtable = soup.select('table', id='AFC')
    NFCtable = soup.select('table', id='NFC')
    AFCStandDf = pd.read_html(str(AFCtable))[0]
    NFCStandDf = pd.read_html(str(NFCtable))[0]
    curr_year = year - count
    AFCStandDf['Year'] = curr_year
    AFCStandDfs.append(AFCStandDf)
    NFCStandDf['Year'] = curr_year
    NFCStandDfs.append(NFCStandDf)    
    count+=1


#initialize Dataframe    
AFCStandings = AFCStandDfs[0]
NFCStandings = NFCStandDfs[0]
for i in range(len(AFCStandDfs)-1):    
    #append DF, reset index to continuous
    AFCStandings = AFCStandings.append(AFCStandDfs[i+1],ignore_index = True)
    NFCStandings = NFCStandings.append(NFCStandDfs[i+1],ignore_index = True)

#drop conference labels (north,east,west,south)    
AFCStandings.drop(AFCStandings.index[0:len(AFCStandings):5], inplace = True)
NFCStandings.drop(NFCStandings.index[0:len(NFCStandings):5], inplace = True)


In [124]:
AFCStandings.head(16)

,Tm,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS,Year
1,Buffalo Bills*,13,3,0,.813,501,375,126,7.9,-0.2,7.7,7.1,0.6,2020
2,Miami Dolphins,10,6,0,.625,404,338,66,4.1,-1.2,3.0,0.3,2.7,2020
3,New England Patriots,7,9,0,.438,326,353,-27,-1.7,0.7,-1.0,-4.2,3.2,2020
4,New York Jets,2,14,0,.125,243,457,-214,-13.4,1.9,-11.5,-8.7,-2.8,2020
6,Pittsburgh Steelers*,12,4,0,.750,416,312,104,6.5,-1.8,4.7,0.3,4.4,2020
7,Baltimore Ravens+,11,5,0,.688,468,303,165,10.3,-2.0,8.3,3.9,4.3,2020
8,Cleveland Browns+,11,5,0,.688,408,419,-11,-0.7,-1.9,-2.6,0.3,-2.8,2020
9,Cincinnati Bengals,4,11,1,.281,311,424,-113,-7.1,-0.4,-7.5,-5.4,-2.1,2020
11,Tennessee Titans*,11,5,0,.688,491,439,52,3.3,-1.7,1.6,4.7,-3.1,2020
12,Indianapolis Colts+,11,5,0,.688,451,362,89,5.6,-2.8,2.8,1.2,1.6,2020


In [127]:
NFCStandings.head(16)

,Tm,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS,Year
1,Buffalo Bills*,13,3,0,.813,501,375,126,7.9,-0.2,7.7,7.1,0.6,2020
2,Miami Dolphins,10,6,0,.625,404,338,66,4.1,-1.2,3.0,0.3,2.7,2020
3,New England Patriots,7,9,0,.438,326,353,-27,-1.7,0.7,-1.0,-4.2,3.2,2020
4,New York Jets,2,14,0,.125,243,457,-214,-13.4,1.9,-11.5,-8.7,-2.8,2020
6,Pittsburgh Steelers*,12,4,0,.750,416,312,104,6.5,-1.8,4.7,0.3,4.4,2020
7,Baltimore Ravens+,11,5,0,.688,468,303,165,10.3,-2.0,8.3,3.9,4.3,2020
8,Cleveland Browns+,11,5,0,.688,408,419,-11,-0.7,-1.9,-2.6,0.3,-2.8,2020
9,Cincinnati Bengals,4,11,1,.281,311,424,-113,-7.1,-0.4,-7.5,-5.4,-2.1,2020
11,Tennessee Titans*,11,5,0,.688,491,439,52,3.3,-1.7,1.6,4.7,-3.1,2020
12,Indianapolis Colts+,11,5,0,.688,451,362,89,5.6,-2.8,2.8,1.2,1.6,2020


In [134]:
#get playoff results data
#includes winner and score

#for loop to get data for X years


year = 2020
#URL for website data
url = "https://www.pro-football-reference.com/years/"
urls = []
PlayOffdfs = []
for index in np.arange(3):
    #create URL for BeautifulSoup
    urls.append(url + str(year - index) +'/')
    years = year - index

count = 0
for index in urls:
    soup = BeautifulSoup(requests.get(index).content, 'html.parser')
    table = soup.select_one('#all_playoff_results').find_next(text=lambda t: isinstance(t, Comment))
    table = BeautifulSoup(table, 'html.parser') 
    PlayOffdf = pd.read_html(str(table))[0]
    curr_year = year - count
    PlayOffdf['Year'] = curr_year
    PlayOffdfs.append(PlayOffdf)
    count+=1


#initialize Playoffs Dataframe    
PlayOffs = PlayOffdfs[0]

for i in range(len(PlayOffdfs)-1):    
    #append DF, reset index to continuous
    PlayOffs = PlayOffs.append(PlayOffdfs[i+1],ignore_index = True)





In [137]:
PlayOffs.head(13)

,Week,Day,Date,Winner/tie,Unnamed: 4,Loser/tie,Unnamed: 6,Pts,Pts.1,Year
0,WildCard,Sat,2021-01-09,Buffalo Bills,NaN,Indianapolis Colts,boxscore,27,24,2020
1,WildCard,Sat,2021-01-09,Los Angeles Rams,@,Seattle Seahawks,boxscore,30,20,2020
2,WildCard,Sat,2021-01-09,Tampa Bay Buccaneers,@,Washington Football Team,boxscore,31,23,2020
3,WildCard,Sun,2021-01-10,Cleveland Browns,@,Pittsburgh Steelers,boxscore,48,37,2020
4,WildCard,Sun,2021-01-10,New Orleans Saints,NaN,Chicago Bears,boxscore,21,9,2020
5,WildCard,Sun,2021-01-10,Baltimore Ravens,@,Tennessee Titans,boxscore,20,13,2020
6,Division,Sat,2021-01-16,Green Bay Packers,NaN,Los Angeles Rams,boxscore,32,18,2020
7,Division,Sat,2021-01-16,Buffalo Bills,NaN,Baltimore Ravens,boxscore,17,3,2020
8,Division,Sun,2021-01-17,Kansas City Chiefs,NaN,Cleveland Browns,boxscore,22,17,2020
9,Division,Sun,2021-01-17,Tampa Bay Buccaneers,@,New Orleans Saints,boxscore,30,20,2020
